In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns

# Data

I performed an open ionbot and msfragger search (oxididation of M variable, max 2 missed cleavages).

See SOP document for more details on how MSFragger was ran.

In [2]:
#set this your local folder as the data was pushed to the repo as well
folder = "/home/compomics/tools/ionbot_paper"
#datasets = ["PXD001468","PXD000561","PXD000815","PXD001250","TMTCPTAC"]
datasets = ["PXD001468"]


 # Results
 
Load ionbot PSM results (first-matches + coeluting matches + features) + protein results.

In [3]:
def contains_missed_cleavage(x):
    if "K" in x[:-1]:
        return True
    if "R" in x[:-1]:
        return True
    return False

def is_in_s(x):
    return x in s

In [9]:
results = {}
for datas in datasets:
    for suffix in ["","_nort","_nocorr","_nocorr_nort","_closed"]:
        dataset = datas + suffix
        print(dataset)
        #Processing dataset
        if "PXD001468" in dataset:
            def get_scan_ionbot(x):
                return int(x.split('.')[1])
            def get_spectrum_file_ionbot(x):
                return x.split('.')[0]
        else:
            def get_scan_ionbot(x):
                print(x)
                return int(x.split('=')[-1])
            def get_spectrum_file_ionbot(x):
                return x.split('.')[0]
        folder2 = folder + "/" + dataset
        ionbot = pd.read_csv("%s/ionbot.first.csv"%folder2)
        ionbot["rank"] = ["first"]*len(ionbot)
        tmp = pd.read_csv("%s/ionbot.lower.csv"%folder2)
        tmp["rank"] = ["lower"]*len(tmp)
        ionbot = pd.concat([ionbot,tmp])
        #NO IDEA WHAT HAPPENED HERE...
        ionbot = ionbot.drop_duplicates("ionbot_match_id")
        tmp = pd.read_csv("%s/ionbot.features.csv"%folder2)
        ionbot = ionbot.merge(tmp,on="ionbot_match_id",how="left")
        if "PXD001468" in dataset:
            ionbot["scan"] = ionbot["spectrum_title"].apply(get_scan_ionbot)
            ionbot["sfile"] = ionbot["spectrum_title"].apply(get_spectrum_file_ionbot)
        else:
            ionbot["sfile"] = ionbot["spectrum_file"].apply(get_spectrum_file_ionbot)
        ionbot["PSM"] = ionbot["scan"].astype(str) + ionbot["sfile"] +  "XYX" +  ionbot["matched_peptide"]
        ionbot = ionbot[(ionbot["database"]=="T")&(ionbot["q-value"]<=0.01)]
        ionbot["contains_missed_cleavage"] = ionbot["matched_peptide"].apply(contains_missed_cleavage)    
        ionbot["num_passed"] = ionbot.groupby(["sfile","scan"])["scan"].transform("count")
        ionbot = ionbot[(ionbot["num_passed"]==1) |
                (ionbot["rank"]=="lower") |
                ((ionbot["num_passed"]>1) & (ionbot["rank"]=="first") & (ionbot["unexpected_modification"].isna()) & (ionbot["contains_missed_cleavage"]==False))]
        #need to re-assign best_psm
        print(ionbot["num_passed"].value_counts())
        best_psms = ionbot.sort_values("psm_score").drop_duplicates(["sfile","scan"], keep="last")
        best_psm_ids = set(best_psms["ionbot_match_id"])
        ionbot["rank"] = ionbot["ionbot_match_id"].apply(
        lambda x: "first" if x in best_psm_ids else "lower")    
        results[dataset] = ionbot


PXD001468
1    676439
2     62279
3      3002
4       102
Name: num_passed, dtype: int64
PXD001468_nort
1    650475
2     58775
3      2647
4        78
Name: num_passed, dtype: int64
PXD001468_nocorr
1    631960
2     46597
3      1858
4        42
Name: num_passed, dtype: int64
PXD001468_nocorr_nort
1    594733
2     41625
3      1493
4        32
Name: num_passed, dtype: int64
PXD001468_closed
1    460648
2      7377
3       252
4        23
5         9
6         5
Name: num_passed, dtype: int64


In [5]:
def is_in_s(x):
    return x in s

to_write = []

for datas in datasets:
    dataset = datas
    #compute statistics
    print(dataset)

    dataset = datas + "_closed"
    ionbot = results[dataset]
    full = len(ionbot)
    print(full)
    to_write.append(["","count","PSM",datas,"_closed",full])
    full = len(ionbot["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_closed",full])

    #compute group FDR

    dataset = datas
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    #ionbot[ionbot["is_7Da"]==True].sort_values("psm_score").to_csv("wide_error_pxd001468.csv",index=False)
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open",unexp])

    dataset = datas + "_nort"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_noR",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_noR",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_noR",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_noR",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_noR",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_noR",unexp])

    dataset = datas + "_nocorr"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_no_corr",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_no_corr",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_no_corr",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_no_corr",unexp])

    dataset = datas + "_nocorr_nort"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_noR_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_noR_no_corr",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_noR_no_corr",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_noR_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_noR_no_corr",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_noR_no_corr",unexp])

    #overlap between open and open without corr and rt information
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    dataset = datas + "_nocorr_nort"
    ionbot = results[dataset]
    s2 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"_open",len(s1.difference(s2))])
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"intersection",len(s1.intersection(s2))])
    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)
    #now I cut the peptide bacause I want the overlap in title
    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled = len(tmp2.intersection(tmp3))
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"_open_noR_no_corr",uniques2-overruled])
    to_write.append(["overruled","overlap_open_noR_no_corr","PSM",datas,"_open_noR_no_corr",overruled])


    #overlap between open and closed
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    dataset = datas + "_closed"
    ionbot = results[dataset]
    s2 = set(ionbot["PSM"])
    to_write.append(["","overlap_closed","PSM",datas,"intersection",len(s1.intersection(s2))])

    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)

    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled = len(tmp2.intersection(tmp3))
    to_write.append(["","overlap_closed","PSM",datas,"_closed",uniques2-overruled])
    to_write.append(["overruled","overlap_closed","PSM",datas,"_closed",overruled])

    dataset = datas
    ionbot = results[dataset+"_D"]
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    dataset = datas + "_closed_D"
    ionbot = results[dataset]
    s2 = set(ionbot["PSM"])

    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)

    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled_decoy = len(tmp2.intersection(tmp3))
    print("overruled closed-open FDR: %f"%(overruled_decoy/overruled))




    dataset = datas
    ionbot = results[dataset]
    s = s1.difference(s2)
    ionbot["unique"] = ionbot["PSM"].apply(is_in_s)
    ionbot = ionbot[ionbot["unique"]==True]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","overlap_closed","PSM",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","overlap_closed","PSM",datas,"_open",wide])
    to_write.append(["is_unexpected","overlap_closed","PSM",datas,"_open",unexp])

    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["matched_peptide"])
    dataset = datas + "_closed"
    ionbot = results[dataset]
    s2 = set(ionbot["matched_peptide"])
    to_write.append(["","overlap_closed","peptide",datas,"_open",len(s1.difference(s2))])
    to_write.append(["","overlap_closed","peptide",datas,"intersection",len(s1.intersection(s2))])
    to_write.append(["","overlap_closed","peptide",datas,"_closed",len(s2.difference(s1))])

to_write.append(["","count","PSM","PXD001468","_open_om",586403])
to_write.append(["","count","PSM","PXD000561","_open_om",227746])
to_write.append(["","count","PSM","PXD001250","_open_om",353869])
to_write.append(["","count","PSM","PXD000815","_open_om",73342])
to_write.append(["","count","PSM","TMTCPTAC","_open_om",367960])
to_write.append(["","count","peptide","PXD001468","_open_om",130485])
to_write.append(["","count","peptide","PXD000561","_open_om",63683])
to_write.append(["","count","peptide","PXD001250","_open_om",82164])
to_write.append(["","count","peptide","PXD000815","_open_om",21853])
to_write.append(["","count","peptide","TMTCPTAC","_open_om",149891])




PXD001468
462664
overruled closed-open FDR: 0.008868


In [6]:
to_write = pd.DataFrame(to_write,columns=["unexpected","type","level","spectrum_file","search","value"])
to_write.to_csv("id_overlap.csv",index=False)

In [7]:
to_write

,unexpected,type,level,spectrum_file,search,value
0,,count,PSM,PXD001468,_closed,462664
1,,count,peptide,PXD001468,_closed,123006
2,,count,PSM,PXD001468,_open,411971
3,is_7Da,count,PSM,PXD001468,_open,156443
4,is_unexpected,count,PSM,PXD001468,_open,138096
5,,count,peptide,PXD001468,_open,30714
6,is_7Da,count,peptide,PXD001468,_open,70422
7,is_unexpected,count,peptide,PXD001468,_open,52376
8,,count,PSM,PXD001468,_open_noR,398686
9,is_7Da,count,PSM,PXD001468,_open_noR,143678


In [8]:
for dataset in datasets:
    print(dataset)
    tmp = to_write[(to_write["spectrum_file"]==dataset)&(to_write["level"]=="PSM")]
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_closed")]["value"].sum()
    v2 = tmp[(tmp["type"]=="overlap_closed")&(tmp["search"]=="_closed")]["value"].sum()
    v = v2 / (v1+v2)
    print("lost in closed: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with RT: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr-RT: %f"%(100*v))
    tmp = to_write[(to_write["spectrum_file"]==dataset)&(to_write["level"]=="peptide")]
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr-RT: %f"%(100*v))



PXD001468
lost in closed: 11.765458
gained with RT: 3.849202
gained with corr: 7.588569
gained with corr-RT: 13.140366
gained with corr-RT: 10.906639
